# REELS: Event optimizer

<img style="float: right;" src="images/piece-puzzle.jpg">

## Event Optimization Overview

This notebook explores how to optimize event representations in `reels` to improve model performance and interpretability. You will learn how to:

* Control event discovery using configuration parameters
* Reduce high-cardinality event spaces
* Evaluate the impact of different event settings on model performance
* Compare optimized and non-optimized configurations

The goal is to show how thoughtful event design can significantly influence both predictive quality and model structure.


## What is Event Optimization?

In `reels`, event optimization refers to the process of refining how raw event codes are represented before model training. Real-world datasets often contain thousands of distinct event codes, many of which are sparse, redundant, or weakly informative.

Event optimization reduces this high-cardinality space by:

* Limiting the maximum number of discovered events
* Selecting the most informative codes
* Aggregating or pruning infrequent events

The result is a more compact and balanced event representation, which can improve model stability, interpretability, and predictive performance while reducing noise introduced by rare or random codes.


## Prerequisite

<img style="float: left;" src="images/reels_small.png"> You should complete the **reels_walkthrough** tutorial before starting with this one!


## Imports

We will use some standard packages in this notebook that we include here.

In [1]:
import random, time, io

import pandas as pd

from sklearn import metrics

Now, we import reels.

In [2]:
import reels

Additionally, we can verify that we are using the right version. (This notebook requires at least version 1.3.1.)

In [3]:
reels.__version__

'1.6.3'

## The problem Event Optimization solves!

Real datasets possibly have hundreds of millions of transactions, millions of clients and thousands of event codes. Reels usually works just fine in those conditions.

On top of that, there may be additional complications:

  - Sequences for each client are typically very long (over 100 events).
  - We want to predict a target that is extremely unbalanced, like in fraud applications where only 1 in 10,000 clients are targets.

Fitting the data will result in way too many sequences being seen just once in the dataset. That will result in extreme overfitting to the point that the model will be of very limited use when applied to unseen data. 

### What is the "magical solution" to that?

In these conditions, we will typically fit the model with the `as_states` argument of `Targets.fit()` set to true. If, rather than having 30,000 event codes, we could 
"magically" select the 20 codes that are relevant to predicting the target and assign a dummy code to all the rest, we would benefit from both:

  - The sequences becoming much shorter (as states)
  - The sequences being much less unique
  
That produces a much smaller tree with many more visits per node that will generalize much better to unseen data.

### What is the "practical solution" to that?

Needless to say, it is hard to "magically" find **the 20** codes. When we limit the discovery of codes by setting the `max_num_events` argument when constructing `Events` objects, we are only limiting the event to the most **frequently seen** in the dataset, **not necessarily relevant** to any prediction.

The good news is: we may find a set of 200 codes that includes 15 of them that works almost as well as the "perfect" fit.

And we can do that, if we are lucky, by calling an automatic search with default parameters. More likely, it will happen using the tools the **event optimizer** provides to better understand the problem and manually including and excluding sets of codes from the model.

### The quest for "The Dictionary"

Summarizing, we want to find a dictionary that maps thousands of codes in an **Events** object into tens of relevant codes.

This notebook provides examples to understand how to discover and apply that dictionary.

## The dataset

This synthetic dataset is "loosely based" on a real BBVA web navigation dataset. The original dataset cannot be used in the tutorial for compliance and practical considerations.

This synthetic dataset is smaller, simpler and much less unbalanced than the original one. Therefore, it works much better "out of the box". Nonetheless, it does improve somewhat with event optimization and, because it is not so easy to auto detect the signal, provides a good basis to understand manual intervention on the optimization algorithm.

The following code defines the function `create_datasets()` that uses a Clips object to read the test sequences from it and modifies them at random.

In [4]:
def build_chunk(seq, cli, time_ori, max_delay):
	N = len(seq)

	emi    = ['click' for _ in range(N)]
	weight = [1 for _ in range(N)]
	client = [cli for _ in range(N)]
	times  = []
	ct = time_ori

	for _ in range(N):
		ct += random.random()*max_delay
		times.append(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ct)))

	return pd.DataFrame(list(zip(emi, seq, weight, client, times)), columns = ['emitter', 'description', 'weight', 'client', 'time'])


def modify_chunk(chunk, cli, reduce, mutate):
	N = chunk.shape[0]

	if N > 5:
		chunk = chunk.sample(frac = reduce, replace = False)
		N = chunk.shape[0]

	chunk.reset_index(drop = True, inplace = True)

	for _ in range(random.randrange(1, round(2 + mutate*N))):
		i = random.randrange(0, N)
		j = random.randrange(0, N)
		x = chunk.description.at[i] + 1
		chunk.description.at[i] = chunk.description.at[j]
		chunk.description.at[j] = x

	chunk.client = [cli for _ in range(N)]

	return chunk


def create_datasets(test = False, n_extra = 10, reduce = 0.95, mutate = 0.15):

	events  = reels.Events()
	clients = reels.Clients()
	clips   = reels.Clips(clients, events)

	chunks = []
	targs  = None

	time_ori  = time.mktime(time.strptime('2022-06-01 00:00:01', '%Y-%m-%d %H:%M:%S'))
	max_delay = 3600

	if test:
		i_base = 400
		i_top  = 500
	else:
		i_base = 0
		i_top  = 400

	for i_cli in range(i_base, i_top):
		seq = clips.test_sequence(i_cli, False)

		cli = 'n%04i%03i' % (i_cli, 0)

		chunk = build_chunk(seq, cli, time_ori, max_delay)
		chunks.append(chunk)

		for t in range(1, n_extra):
			cli	  = 'n%04i%03i' % (i_cli, t)
			chunk = modify_chunk(chunk, cli, reduce, mutate)
			chunks.append(chunk)

		seq = clips.test_sequence(i_cli, True)

		cli = 't%04i%03i' % (i_cli, 0)

		chunk = build_chunk(seq, cli, time_ori, max_delay)
		chunks.append(chunk)

		ct = time.mktime(time.strptime(chunk.time.iloc[-1], '%Y-%m-%d %H:%M:%S'))
		ct += random.random()*max_delay
		ct -= random.random()*max_delay

		chunk = pd.DataFrame([[cli, time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ct))]], columns = ['client', 'time'])
		targs = chunk if targs is None else pd.concat([targs, chunk])

	return pd.concat(chunks), targs


## A simple analysis

We also copy from the **reels_walkthrough** tutorial the function `analyze()` that will serve to evaluate our model.

In [5]:
def analyze(targets, clips, targs):
    target_hashes = set([clients.hash_client_id(str(id)) for id in targs.client])  # This is the set of all the clients who are targets
    Y_obs = [int(hh in target_hashes) for hh in clips.clips_client_hashes()]       # This is the observed target/no_target for all the clients
    
    T = [t for t in targets.predict_clips(clips)]                                  # These are the predicted times
    
    t_copy = T.copy()
    t_copy.sort()
    t_cut = t_copy[sum(Y_obs)]                                                     # t_cut is a cutting time that generates the same number of targets.
    
    Y_pred = [int(t <= t_cut) for t in T]                                          # This is the predicted target/no_target for all the clients
    
    x_tab = pd.crosstab(pd.array(Y_obs), pd.array(Y_pred), rownames = ['Obs'], colnames = ['Pred'])
    
    acc    = metrics.accuracy_score(Y_obs, Y_pred)                                 # We compute basic metrics
    prec   = metrics.precision_score(Y_obs, Y_pred)
    f1     = metrics.f1_score(Y_obs, Y_pred)

    print(x_tab)
    print('Accuracy: %.3f, precision: %.3f, f1-score: %.3f' % (acc, prec, f1))

We create the datasets.

In [6]:
train, train_targs = create_datasets(False)
test,  test_targs  = create_datasets(True)

In [7]:
train

,emitter,description,weight,client,time
0,click,1,1,n0000000,2022-06-01 00:45:29
1,click,17,1,n0000000,2022-06-01 01:27:22
2,click,92,1,n0000000,2022-06-01 01:35:07
3,click,227,1,n0000000,2022-06-01 01:59:50
4,click,864,1,n0000000,2022-06-01 02:38:04
...,...,...,...,...,...
14,click,459,1,t0399000,2022-06-01 06:56:36
15,click,459,1,t0399000,2022-06-01 07:53:54
16,click,459,1,t0399000,2022-06-01 08:12:40
17,click,459,1,t0399000,2022-06-01 08:40:08


In [8]:
train_targs

,client,time
0,t0000000,2022-06-01 15:49:23
0,t0001000,2022-06-01 23:37:44
0,t0002000,2022-06-01 03:36:12
0,t0003000,2022-06-01 18:36:42
0,t0004000,2022-06-01 04:13:49
...,...,...
0,t0395000,2022-06-01 08:47:12
0,t0396000,2022-06-02 10:22:26
0,t0397000,2022-06-01 02:55:59
0,t0398000,2022-06-02 12:17:13


And now, we perform a very simple cross-validated fit/predict as we learned in the **reels_walkthrough** tutorial.

In [9]:
intake_train       = reels.Intake(train)
intake_train_targs = reels.Intake(train_targs)
intake_test        = reels.Intake(test)
intake_test_targs  = reels.Intake(test_targs)

</br>

In [10]:
events = reels.Events(max_num_events = 10000)

intake_train.insert_rows(events)
intake_test.insert_rows(events)

events

reels.Events object with 1968 events

</br>

In [11]:
clients = reels.Clients()

clients

Empty reels.Clients object (Empty objects select ALL clients.)

</br>

In [12]:
train_clips = reels.Clips(clients, events)
intake_train.scan_events(train_clips)

train_clips

reels.Clips object with 4130 clips totalling 138716 events

</br>

In [13]:
targets = reels.Targets(train_clips)
intake_train_targs.insert_targets(targets)
targets.fit(agg = 'longest', depth = 100, as_states = True)

targets

reels.Targets object with 4130 clips

Has 400 targets.

Is fitted with 4129 clips.

We evaluate the prediction over the **training set**.

In [14]:
analyze(targets, train_clips, train_targs)

Pred     0    1
Obs            
0     3567  163
1      162  238
Accuracy: 0.921, precision: 0.594, f1-score: 0.594


</br>

In [15]:
test_clips = reels.Clips(clients, events)
intake_test.scan_events(test_clips)

test_clips

reels.Clips object with 1037 clips totalling 30812 events

And evaluate the prediction over the **test set**.

In [16]:
analyze(targets, test_clips, test_targs)

Pred    0   1
Obs          
0     860  77
1      72  28
Accuracy: 0.856, precision: 0.267, f1-score: 0.273


## Event Optimization

So far, we have trained a rather good model that performs worse on test data, just as expected.

We can also check some statistics about the nodes of the fitted tree.

In [17]:
print(targets.describe_tree())

tree.size() : 95158

num_of_nodes_with_zero_visits : 0
num_of_no_targets_one_visit   : 79616
num_of_has_target_one_visit   : 10009
num_of_no_targets_more_visits : 5121
num_of_has_target_more_visits : 412
num_of_no_targets_final_node  : 3700
num_of_has_target_final_node  : 373



As we would expect, it has many nodes with only one visit and is somewhat unbalanced (about 8:1).

In case the original data provides some business insight to us, since we assume the codes are just meaningless numbers,
we are going to build a simple dictionary mapping the content to the code. In this case, we map the field `description` (that in our example contains just numbers, but could have been descriptive text.)

Later we will add a column to our code exploration dataset using this.

This can be done by just iterating through the object via `describe_events()`.

In [18]:
description = {}
for emitter, descr, weight, code in events.describe_events():
    description[code] = descr

Now, we make a copy of the events dataset and optimize the copy, since the optimization will alter the object and we are just running it to get some data.

In [19]:
events_copy = events.copy()

Now we run the `optimize_events()` method, just for one iteration and removing exponential decay and the confidence interval.

The **exponential decay** is a parameter that allows weighting the score of a code less as it is found deeper in the tree.

The **confidence interval** is computed exactly like in `Targets().fit()` and allows computing lift based on the lower bound of a binomial confidence interval for a proportion rather than the proportion itself.

In [20]:
success, dictionary, top_codes, log = events_copy.optimize_events(train_clips, targets, num_steps = 1, exp_decay = 0, lower_bound_p = 0)

It returns a tuple of 4 elements:

 1. the dictionary that we are not going to use yet, 
 2. a boolean that is true on success,

In [21]:
assert success

 3. a log saved as a string giving us some idea about how the search went,

In [22]:
print(log)

Preprocessing:

  1829 codes found in clips.
  139 codes removed from internal EventMap.
  Current score = 0.845005

Step 1 of 1

  Trying:
    Code 1740 as 2
    Code 1690 as 3
    Code 1638 as 4
    Code 1366 as 5
    Code 1456 as 6
    ---------------
    Score = 0.075123
    Best score so far.

== F I N A L ==

  Final score      = 0.075123


 4. and `top_codes`, a dataframe (in csv format) with the performance of each code.
    
We convert it into a pandas dataframe by passing the string to `pd.read_csv()` using an `io.StringIO` and sort it by the column `n_incl_target`.

In [23]:
top = pd.read_csv(io.StringIO(top_codes), sep = '\t')
top['description'] = [description[c] for c in top.code]
top.sort_values(by = 'n_incl_target', ascending = False, inplace = True)
top.reset_index(drop = True, inplace = True)

top

,n_succ_seen,n_succ_target,n_incl_seen,n_incl_target,sum_dep,n_dep,edf,prop_succ,prop_incl,lift,score,code,description
0,22378,2600,15472,2030,596335,13151,1.0,0.116186,0.131205,0.755779,0.099162,1,1
1,14563,1797,6877,1022,254176,5881,1.0,0.123395,0.148611,0.790435,0.117468,2,17
2,11953,1364,5076,654,140533,4067,1.0,0.114114,0.128842,0.755683,0.097363,20,349
3,16877,1452,8851,549,374802,7635,1.0,0.086034,0.062027,0.542880,0.033673,10,2
4,10761,1128,3127,310,124557,2818,1.0,0.104823,0.099137,0.665649,0.065990,17,350
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1824,4129,399,1,0,1,1,1.0,0.096634,0.000000,0.000000,0.000000,513,2218
1825,2,0,2,0,16,2,1.0,0.000000,0.000000,0.000000,0.000000,514,2219
1826,4822,496,19,0,531,18,1.0,0.102862,0.000000,0.000000,0.000000,524,238
1827,4226,421,14,0,245,13,1.0,0.099621,0.000000,0.000000,0.000000,525,343


NOTE: We have added the field `description` from the original dataset in case it is more informative than the field code.

The first four columns are the number of times seen vs target for both the node with the code (called "incl" for included) and for the parent node. The parent node is called "succ" (successor) because the tree is built in reverse time order. Therefore, the successor in the time sequence is the parent in the tree.

The next two, `sum_dep` and `n_dep` provide an idea of how deep the code is found on average (== sum_dep/n_dep) in the sequence and can be used in the score when exponential decay is not zero.

The next is `edf` (exponential decay factor) and multiplies the final score. E.g. If we set `exponential_decay` to 0.00693 it decays to approx 0.5 in 100 steps because (1- 0.00693)^100 is 0.4988.

The next two are the proportion (or lower bound for the confidence interval) of target/seen for both "incl" and "succ" as explained above.

The lift is the quotient of the two and can be log transformed depending on the argument `log_lift`.

And the final score is the product `edf*prop_incl*lift`.

This score is used by a greedy algorithm to select the codes that are considered for inclusion in decreasing score order.

### Including/excluding codes

We can use this dataset to manually include and exclude. Let's suppose we want all the codes with more than 100 targets included so that we don't risk leaving them out.
We may also want to exclude every code that has never been part of a target. 

In [24]:
include = ','.join([str(x) for x in top.loc[top['n_incl_target'] > 100, 'code']])

In [25]:
exclude = ','.join([str(x) for x in top.loc[top['n_incl_target'] < 1, 'code']])

<div class="alert alert-block alert-warning">
<b>Important:</b> Make sure you only pass comma separated strings of integers to the arguments <b>force_include</b> and <b>force_exclude</b>. A wrong type may crash your program.

Again, we make a copy and work with the copy since we don't want to modify `events` and the previous `events_copy` has been modified.

In [26]:
events_copy = events.copy()

And we call `optimize_events()` again. This time with the included and excluded codes, 20 steps and trying to introduce 4 codes at each step. We leave the default arguments but set threshold to zero to accept any new code that results in improvement no matter how small.

In [27]:
success, dictionary, top_codes, log = events_copy.optimize_events(train_clips, targets, num_steps = 20, codes_per_step = 4, 
                                                                  force_include = include, force_exclude = exclude, threshold = 0, lower_bound_p = 0)

In [28]:
assert success

In [29]:
print(log)

Preprocessing:

  1829 codes found in clips.
  139 codes removed from internal EventMap.
  Current score = 0.845005

Step 1 of 20

  Trying:
    Code 1638 as 34
    Code 1366 as 35
    Code 1690 as 36
    Code 1740 as 37
    ---------------
    Score = 0.714938
    Best score so far.

Step 2 of 20

  Trying:
    Code 1456 as 38
    Code 481 as 39
    Code 980 as 40
    Code 1146 as 41
    ---------------
    Score = 0.719940
    Best score so far.

Step 3 of 20

  Trying:
    Code 919 as 42
    Code 1138 as 43
    Code 1398 as 44
    Code 1436 as 45
    ---------------
    Score = 0.717440
    Threshold (0.000000) not met (diff = -0.002500)

Step 4 of 20

  Trying:
    Code 334 was excluded by the caller
    Code 253 was excluded by the caller
    Code 1277 as 46
    Code 1580 as 47
    Code 613 was excluded by the caller
    Code 494 as 48
    Code 1102 as 49
    ---------------
    Score = 0.722442
    Best score so far.

Step 5 of 20

  Trying:
    Code 359 was excluded by the calle

### How to apply the dictionary

For the sake of this tutorial, we have obtained a dictionary that provides improvement. Of course, in a real case we can iterate and try different things combining the business insight provided by the description with the empirical data from the tree until we get our best results.

At the moment, `events_copy` is already optimized and could be used. 

An alternative method is creating one by copying the original `events` and providing the dictionary to the `copy()` method.

In [30]:
op_events = events.copy(dictionary)

And we use this `op_events` to create new clips and fit a new model.

In [31]:
train_clips = reels.Clips(clients, op_events)
intake_train.scan_events(train_clips)

train_clips

reels.Clips object with 4130 clips totalling 138716 events

</br>

In [32]:
targets = reels.Targets(train_clips)
intake_train_targs.insert_targets(targets)
targets.fit(agg = 'longest', depth = 100, as_states = True)

targets

reels.Targets object with 4130 clips

Has 400 targets.

Is fitted with 4129 clips.

We can also check that the new tree has less nodes overall and many with fewer than one visit.

In [33]:
print(targets.describe_tree())

tree.size() : 57741

num_of_nodes_with_zero_visits : 0
num_of_no_targets_one_visit   : 45329
num_of_has_target_one_visit   : 6900
num_of_no_targets_more_visits : 4949
num_of_has_target_more_visits : 563
num_of_no_targets_final_node  : 2892
num_of_has_target_final_node  : 327



</br>

In [34]:
analyze(targets, train_clips, train_targs)

Pred     0    1
Obs            
0     3477  253
1      223  177
Accuracy: 0.885, precision: 0.412, f1-score: 0.427


</br>

In [35]:
test_clips = reels.Clips(clients, op_events)
intake_test.scan_events(test_clips)
test_clips

reels.Clips object with 1037 clips totalling 30266 events

<br/>

In [36]:
analyze(targets, test_clips, test_targs)

Pred    0   1
Obs          
0     858  79
1      70  30
Accuracy: 0.856, precision: 0.275, f1-score: 0.287


<div class="alert alert-block alert-success">
<b>Note that:</b> The optimized model is smaller, builds a smaller tree and generalizes better on unseen data.
</div>

## Results Interpretation

The comparison between the baseline configuration and the optimized event configuration shows clear structural and performance differences.

When event discovery is unrestricted, the model builds a larger and more fragmented tree. Many nodes are supported by very few visits, which leads to sparsity and less reliable estimates. This is reflected in a more unbalanced structure and weaker generalization performance.

After applying event optimization (e.g., limiting the number of discovered codes), the event space becomes more compact. The resulting tree is smaller, with more balanced node visits and stronger statistical support per split. Performance metrics improve or become more stable across cross-validation folds, indicating better generalization.

In practical terms, these results show that reducing event cardinality does not simply shrink the model — it restructures it. The optimized configuration concentrates signal into fewer, more informative event codes, leading to a cleaner model structure and more reliable predictions.


## Key Takeaways

By the end of this notebook, you should understand that:

* **Event representation strongly influences model behavior.** The way raw codes are defined and limited directly affects tree structure, sparsity, and predictive stability.
* **High-cardinality event spaces can hurt generalization.** Too many rare or weakly supported codes lead to fragmented trees and unreliable estimates.
* **Event optimization is a structural decision, not just a tuning trick.** Limiting or selecting events reshapes the model and concentrates signal into more informative splits.
* **Simpler event spaces can improve both interpretability and performance.** A more compact representation often yields more balanced nodes and more stable cross-validated results.

The main lesson is that thoughtful event design is a core part of modeling with `reels`, not a secondary refinement step.
